# DS Unit 3 Advanced Assignment (Build Week)

Welcome to your first day working at ACME Corp! You are replacing an employee who just got an amazing job offer at Google on the condition that he resign at ACME corp immediately and give no notice (they're fierce competitors of ours).

This has left us in a tough spot here at ACME. The person you are replacing was working on a ML-fueled Recommender System Web App built using song data from Spotify. We promised the client that we would present a basic working prototype of the app to them in just 3 days time, but the project's only half finished! We need you to dive into the code base and see if you can get a basic version of this working before the looming deadline. This is why we pay you the big bucks!

## What you'll find in this repository

- `Spotify_Recommender.ipynb` - This file holds a IPython notebook where the last guy was working on training the machine learning model for the recommender system using a K-Nearest Neighbors Algorithm from Scikit-Learn.

- A Flask app called `spotify` that has a basic interface, but doesn't have any defined database models, doesn't have any song data added to the database, and doesn't have the Machine Learning Model added to it yet!

## Next Steps?

If you could do the following it would really save our bacon:

- Check the Machine Learning model to see if it's making good recommendations; If it's not, let's get the app working first so that we at least have something to show the client and we'll work on improving the ML model if we have time.

- We need to move all of the song data from the CSV dataset into the SQLite database so that we can query the database when we want to display the song recommendations.

- We need to export the ML model from the .ipynb file and include it in the flask app somehow. I've never done it before, but the last guy said something about using a Python Package called Joblib and he shared with me these links a few days back when we were talking about it over Slack:

  - <https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/>

  - <https://maze-runner.medium.com/deploying-the-ml-model-to-the-flask-with-joblib-53f313d24003>

  - <https://www.analyticsvidhya.com/blog/2020/04/how-to-deploy-machine-learning-model-flask/>

- When a person submits a Song Title, we should find that song in the database, send its data through the KNN model to find the row ids of the five most similar songs and then query the database again to get the data of the five recommended songs, and then display that data on the HTML page. There is some fake data in the app that's currently serving as a stand-in for the real data. Once we've got these pieces stitched together we'll want to replace that fake data with the real data so that the real song recommendations will show on the web page when a user submits the form.

Feel free to change any of the files in the app even the HTML files if you feel like you need to. Do whatever you've got to do. I trust your judgement. That's why I hired you.

## If you have time (stretch goals)

If you can get everything I've listed above working then you're just amazing, but let's use all of the time available to make this as impressive as possible for the client. Here's some ideas of other improvements that could be made. I'll leave it up to you to prioritize these.

- Improve the machine learning model so that it's making better recommendations.
  - Maybe we could improve the categorical encoding of the categorical variables in the dataset.
  - Could standardizing the data with StandardScaler help?
  - I bet that the year in which the song was written is a really important piece of information when it comes to making quality recommendations. Can we engineer any features to emphasize the most important columns of data?
  - Should we use a sklearn pipeline somewhere to improve our code's organization?
  - Are there any hyperparameters that we could tune?
  - Have we even started with the right dataset? Maybe there's a better dataset out there that we could use to train the model.

- Is there anything that could be improved with the look and feel of the site like the structure of the HTML or the CSS Styling? I know you don't have very much experience with that, but hey, if we've got time, maybe we give it a shot.

- Can we get the site deployed to Heroku so that we can share a link to the app directly with the client rather than having to show them the development version on one of our computers?

- Maybe we could use the [Spotify API](https://developer.spotify.com/documentation/web-api/) to get additional data about the songs that we have. If we could get information about the song's genre, I bet that would really help us make better recommendations!


# Spotify Recommender System

Dataset: <https://www.kaggle.com/datasets/rodolfofigueroa/spotify-12m-songs>

If you're working in Google Colab it will take a few minutes to upload this dataset. It will be faster if you're able to work in a local Jupyter notebook (By the way, VS Code can run notebooks. Try saving a file with the extension `.ipynb` and then open it in VS Code). 

You might want to subset this dataset to something like 100k rows right off of the bat so that it's easier to work with. You can do all of your modeling with the 100k row version and then once you've got things working the way you want them to you can run the notebook once with the entire dataset. 

In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

In [2]:
# Load dataset and sample it down to 8% of the original size
# Reset index after sampling to make indices easier to reason about
df = pd.read_csv('tracks_features.csv')
df = df.sample(frac=.08, random_state=42).reset_index()

# Drop old index to avoid confusing it for the new one
# df = df.drop(columns=['index'])

### Usable Columns?

Looking at columns ready to use for a basic recommendation system. I could improve this if I make more columns or feature engineer more columns, but I want to get to a working prototype as fast as possible.

- explicit
- danceability
- energy
- key
- loudness
- mode
- speechiness
- acousticness
- time_signature
- year 

Todo:

- Check for Null Values (None)
- Categorically encode `explicit` column

In [3]:
df.shape

(96322, 25)

In [4]:
df.columns

Index(['index', 'id', 'name', 'album', 'album_id', 'artists', 'artist_ids',
       'track_number', 'disc_number', 'explicit', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms',
       'time_signature', 'year', 'release_date'],
      dtype='object')

In [5]:
df.head()

,index,id,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,release_date
0,54449,1aGS6nf2xgv3Xzdob4eOO3,Smokin' Sticky Sticky,Beat'n Down Yo Block,5ZO72kl3xMRRzlpod55k1Q,['Unk'],['0PGtMx1bsqoCHCy3MB3gXA'],15,1,True,...,0.4020,0.002100,0.000000,0.0691,0.422,87.988,380427,4.0,2006,2006-10-03
1,127106,0fJfoqHIIiET2EcgjOfntG,Holding Back the Years,Holding Back The Years,7sV4kCqQYt8agM5TjkdOYU,['Norm Douglas'],['4kxKyoiYhldUlnfeCZtD0D'],1,1,False,...,0.0333,0.316000,0.775000,0.0993,0.880,170.082,266520,4.0,2008,2008-06-13
2,11213,0V2R2LC8dR7S0REieXRaGt,All Along The Watchtower - Live - 1991,"Back On The Bus, Y'All",3jmmx4jRkul3POEhn1cgwF,['Indigo Girls'],['4wM29TDTr3HI0qFY3KoSFG'],7,1,False,...,0.0379,0.709000,0.000000,0.9390,0.430,90.648,383773,4.0,1991,1991-06-04
3,369031,4VUHYLocWOJ2GfvP78AmSs,Windmills,Total Folklore,5PyLkzuxmT6EoVNZCg8Iya,['Dan Friel'],['4HKTPJw50BFASrfhJEHIVP'],2,1,False,...,0.1090,0.000049,0.838000,0.2850,0.594,113.345,82493,4.0,2013,2013-02-19
4,544989,4m8a1AtmCnoeRzSYoQ0oX0,Overnite Flite,Normal Human Feelings,623VIdYR6Y0NCN9yPbMAC6,['Little Suns'],['5OLcAqMbHpecNOIQyTduQ7'],2,1,False,...,0.0323,0.427000,0.000105,0.1970,0.424,127.941,230667,1.0,2013,2013-10-08


In [6]:
# no null values
df.isnull().sum()

index               0
id                  0
name                0
album               0
album_id            0
artists             0
artist_ids          0
track_number        0
disc_number         0
explicit            0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
duration_ms         0
time_signature      0
year                0
release_date        0
dtype: int64

In [7]:
# any column that contains True and False will automatically
# change to 1s and 0s when cast to the `int` datatype
df['explicit'] = df['explicit'].astype(int)

df.head()

,index,id,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,release_date
0,54449,1aGS6nf2xgv3Xzdob4eOO3,Smokin' Sticky Sticky,Beat'n Down Yo Block,5ZO72kl3xMRRzlpod55k1Q,['Unk'],['0PGtMx1bsqoCHCy3MB3gXA'],15,1,1,...,0.4020,0.002100,0.000000,0.0691,0.422,87.988,380427,4.0,2006,2006-10-03
1,127106,0fJfoqHIIiET2EcgjOfntG,Holding Back the Years,Holding Back The Years,7sV4kCqQYt8agM5TjkdOYU,['Norm Douglas'],['4kxKyoiYhldUlnfeCZtD0D'],1,1,0,...,0.0333,0.316000,0.775000,0.0993,0.880,170.082,266520,4.0,2008,2008-06-13
2,11213,0V2R2LC8dR7S0REieXRaGt,All Along The Watchtower - Live - 1991,"Back On The Bus, Y'All",3jmmx4jRkul3POEhn1cgwF,['Indigo Girls'],['4wM29TDTr3HI0qFY3KoSFG'],7,1,0,...,0.0379,0.709000,0.000000,0.9390,0.430,90.648,383773,4.0,1991,1991-06-04
3,369031,4VUHYLocWOJ2GfvP78AmSs,Windmills,Total Folklore,5PyLkzuxmT6EoVNZCg8Iya,['Dan Friel'],['4HKTPJw50BFASrfhJEHIVP'],2,1,0,...,0.1090,0.000049,0.838000,0.2850,0.594,113.345,82493,4.0,2013,2013-02-19
4,544989,4m8a1AtmCnoeRzSYoQ0oX0,Overnite Flite,Normal Human Feelings,623VIdYR6Y0NCN9yPbMAC6,['Little Suns'],['5OLcAqMbHpecNOIQyTduQ7'],2,1,0,...,0.0323,0.427000,0.000105,0.1970,0.424,127.941,230667,1.0,2013,2013-10-08


## Create X Matrix of numeric song attributes

In [8]:
usable_columns =['explicit', 'danceability', 'energy', 'key', 'loudness', 
        'mode', 'speechiness', 'acousticness', 'time_signature', 'year' ]

X = df[usable_columns]

X.head()

,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,time_signature,year
0,1,0.623,0.736,11,-3.657,0,0.4020,0.002100,4.0,2006
1,0,0.585,0.639,2,-9.641,0,0.0333,0.316000,4.0,2008
2,0,0.331,0.466,9,-14.287,0,0.0379,0.709000,4.0,1991
3,0,0.193,0.856,4,-2.970,1,0.1090,0.000049,4.0,2013
4,0,0.546,0.373,3,-13.929,1,0.0323,0.427000,1.0,2013


# Use Nearest Neighbors to get 5 most similar songs.

> Indented block



In [27]:
neigh = NearestNeighbors(n_neighbors=5, n_jobs=-1)
neigh.fit(X)

NearestNeighbors(n_jobs=-1)

In [25]:
neigh.__dict__

{'n_neighbors': 5,
 'radius': 1.0,
 'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'p': 2,
 'n_jobs': -1,
 'n_features_in_': 10,
 'effective_metric_params_': {},
 'effective_metric_': 'euclidean',
 '_fit_method': 'kd_tree',
 '_fit_X': array([[1.000e+00, 6.230e-01, 7.360e-01, ..., 2.100e-03, 4.000e+00,
         2.006e+03],
        [0.000e+00, 5.850e-01, 6.390e-01, ..., 3.160e-01, 4.000e+00,
         2.008e+03],
        [0.000e+00, 3.310e-01, 4.660e-01, ..., 7.090e-01, 4.000e+00,
         1.991e+03],
        ...,
        [0.000e+00, 6.870e-01, 5.600e-01, ..., 3.880e-01, 4.000e+00,
         2.004e+03],
        [0.000e+00, 2.010e-01, 9.220e-03, ..., 9.350e-01, 4.000e+00,
         2.003e+03],
        [0.000e+00, 6.750e-01, 6.350e-01, ..., 7.860e-01, 4.000e+00,
         2.003e+03]]),
 'n_samples_fit_': 96322,
 '_tree': <sklearn.neighbors._kd_tree.KDTree at 0x1d8ff380b80>}

In [10]:
# Track name needs to be exact match of spelling, punctuation and capitalization
track_name = "Holding Back the Years"

# Look at the song that we want to find recommendations for
df[df['name'] == track_name]

,index,id,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,release_date
1,127106,0fJfoqHIIiET2EcgjOfntG,Holding Back the Years,Holding Back The Years,7sV4kCqQYt8agM5TjkdOYU,['Norm Douglas'],['4kxKyoiYhldUlnfeCZtD0D'],1,1,0,...,0.0333,0.316,0.775,0.0993,0.880,170.082,266520,4.0,2008,2008-06-13
82635,259804,5F9WGLNnZRRwVyiCt1nHDr,Holding Back the Years,The Lost and Found,4fZx2cNk1Vod8jZkPSWBpv,['Gretchen Parlato'],['76Gi1qoWLrIerL5FcL0TZb'],1,1,0,...,0.0427,0.778,0.130,0.1150,0.127,92.624,226587,4.0,2011,2011-04-05


In [18]:
df[df['name'] == track_name].index[0]

1

In [11]:
# We may have multiple tracks that match this title, we'll just select the first one
# We'll grab only its row index and then use that select the corresponding song's
# data from our X matrix.
track_index = df[df['name'] == track_name].index[0]

track_data = X.iloc[track_index]

track_data

explicit             0.0000
danceability         0.5850
energy               0.6390
key                  2.0000
loudness            -9.6410
mode                 0.0000
speechiness          0.0333
acousticness         0.3160
time_signature       4.0000
year              2008.0000
Name: 1, dtype: float64

In [12]:
# Input to model must be a 2D array
# .reshape(1,-1) turns a 1D array into a 2D array
# (basically just adds an extra set of square brackets at
# the beginning and end of the array.)
track_data = track_data.values.reshape(1,-1)

track_data

array([[ 0.000e+00,  5.850e-01,  6.390e-01,  2.000e+00, -9.641e+00,
         0.000e+00,  3.330e-02,  3.160e-01,  4.000e+00,  2.008e+03]])

In [16]:
# Since the selected song is also in the training data,
# the most similar song is itself 
# We will ask for 6 songs to get back 5 songs in addition to the one provided
distances, song_indexes = neigh.kneighbors(track_data, 6)

song_indexes

array([[    1, 69557, 34753,  7222, 50380, 37367]], dtype=int64)

In [17]:
# 5 most similar songs
for index in song_indexes:
  print(df.iloc[index][['name', 'artists']])

                            name                  artists
1         Holding Back the Years         ['Norm Douglas']
69557                 Ma Mélodie           ['Feet Peals']
34753     Minha Bênção (Ao Vivo)  ['Padre Marcelo Rossi']
7222   Stuck In A Glass Elevator           ['The Myriad']
50380        Club Hip Hop Beat 2       ['Jorge Quintero']
37367           Like...monk-like    ['The Reese Project']
